## MiniCaML: Costrutto try-with


Estendere l'interprete visto a lezione per implementare una rudimentale gestione delle eccezioni:
 - Definire un insieme di eccezioni, contenente almeno `VarUnbound` e `NonBooleanGuard`
 - Definire un costrutto try-with che permetta all'utente di restituire un valore alternativo quando viene sollevata un'eccezione
 
 
 Esempio (in Ocaml):
 ```
 let safe_inverse n = 
     try 
         1 / n
     with
         Division_by_zero -> 0
 ```

## Soluzione:

Quando estendiamo un interprete per implementare un nuovo costrutto del linguaggio, nella maggior parte dei casi si procende in 2 step: 
- **Estendere la sintassi**: In questo caso creando un nuovo tipo di espressione nell'Albero della sintassi astratta
- **Estendere la semantica**: In questo caso creando nuovi valori evT che rappresentino i vari errori

### Tipi ed Espressioni

Estendiamo la sintassi aggiungendo un nuovo nodo all'AST, il nodo `TryWith`.
`TryWith` è composto da 
- un'espressione `e1`, che potrebbe sollevare una eccezione quando viene valutata
- una eccezione `excep`
- una espressione `e2`, che va valutata quando viene sollevata una eccezione `exec` durante la valutazione di `e1`

Definiamo `TryWith` come altro costruttore in `exp` e definiamo un altro tipo variant `my_exception` per contenere 3 diversi tipi di eccezioni: Variabili non definite, Costrutti if con una guardia non booleana, e Divisioni per zero. Questi eccezioni sono solo **elementi sintattici**, rappresentano quello che l'utente scriverebbe dopo la keyword `with` in MiniCaml.

In [25]:
(* Identificatori *)
type ide = string;;

(* I tipi *)
type tname =  TInt | TBool | TString | TClosure | TRecClosure | TUnBound


(* Eccezioni, built-in nel linguaggio, non definibili dall'utente di MiniCaml*)
type my_exception = ExVarUnbound | ExNonBoolGuard | ExDivByZero

(* Abstract Expressions = espressioni nella sintassi astratta, 
   compongono l'Albero di Sintassi Astratta *)
type exp = 
    | EInt of int
    | CstTrue 
    | CstFalse
    | EString of string
    | Den of ide
    (* Operatori binari da interi a interi*)
    | Sum of exp * exp
    | Diff of exp * exp
    | Prod of exp * exp
    | Div of exp * exp
    (* Operatori da interi a booleani *)
    | IsZero of exp
    | Eq of exp * exp
    | LessThan of exp*exp
    | GreaterThan of exp*exp
    (* Operatori su booleani *)
    | And of exp*exp
    | Or of exp*exp
    | Not of exp
    (* Controllo del flusso, funzioni *)
    | IfThenElse of exp * exp * exp
    | Let of ide * exp * exp
    | Letrec of ide * ide  * exp * exp
    | Fun of ide * exp
    | Apply of exp * exp
    (*Costrutto try-with*)
    | TryWith of exp*my_exception*exp

type ide = string


type tname = TInt | TBool | TString | TClosure | TRecClosure | TUnBound


type my_exception = ExVarUnbound | ExNonBoolGuard | ExDivByZero


type exp =
    EInt of int
  | CstTrue
  | CstFalse
  | EString of string
  | Den of ide
  | Sum of exp * exp
  | Diff of exp * exp
  | Prod of exp * exp
  | Div of exp * exp
  | IsZero of exp
  | Eq of exp * exp
  | LessThan of exp * exp
  | GreaterThan of exp * exp
  | And of exp * exp
  | Or of exp * exp
  | Not of exp
  | IfThenElse of exp * exp * exp
  | Let of ide * exp * exp
  | Letrec of ide * ide * exp * exp
  | Fun of ide * exp
  | Apply of exp * exp
  | TryWith of exp * my_exception * exp


### Ambiente

Passiamo ora al secondo step, la semantica. Come dovrebbe essere valutata una espressione `TryWith(e1, excep, e2)`? Per gestire le eccezioni secondo l'approccio funzionale, possiamo introdurre dei **valori semantici** di errore, oltre a `UnBound`, quali `NonBoolGuard` e `DivByZero`. Ci aspettiamo che l'espressione `Div(EInt(42), EInt(2))` venga valutata al valore `EInt(21)`, mentre l'espressione `Div(EInt(42), EInt(0))` venga valutata al valore `DivByZero`. (Questo è lo stesso approccio seguito nell'aritmetica a virgola mobile IEEE 754, dove dividento un numero per zero viene restituita una particolare sequenza binaria, interpretata come `NaN`, Not a Number).

Quindi una espressione, quando è presente un errore, viene valutata a un valore di errore come quelli elencati. Diventa allora immediato definire la semantica di `TryWith(e1, excep, e2)`: Se il risultato di `e1` è un valore "normale", allora va restituito quel valore; se invece il risultato di `e2` è un valore di errore, allora bisogna valutare `e2` se e solo se l'eccezione (semantica) incontrata coincide con l'eccezione (sintattica) `excep`.

In [26]:
(* Ambiente polimorfo *)
type 't env = ide -> 't

(* Evaluation types = tipi  esprimibili *)
type evT = 
    | Int of int 
    | Bool of bool 
    | String of string 
    | Closure of ide * exp * evT env 
    | RecClosure of ide * ide * exp * evT env
    | UnBound
    | NonBoolGuard
    | DivByZero

(* Ambiente vuoto *)
let emptyenv = function x -> UnBound

(* Binding fra una stringa x e un valore primitivo evT *)
let bind (s: evT env) (x: ide) (v: evT) = 
    function (i: ide) -> if (i = x) then v else (s i)

type 't env = ide -> 't


type evT =
    Int of int
  | Bool of bool
  | String of string
  | Closure of ide * exp * evT env
  | RecClosure of ide * ide * exp * evT env
  | UnBound
  | NonBoolGuard
  | DivByZero


val emptyenv : 'a -> evT = <fun>


val bind : evT env -> ide -> evT -> ide -> evT = <fun>


### Typechecking

Non serve fare alcuna modifica alla funzione di typechecking.

In [27]:
(* Type-checking *)
let typecheck ((x, y) : (tname*evT)) = 
    match x with
    | TInt -> (match y with 
               | Int(u) -> true
               | _ -> false
               )
    | TBool -> (match y with 
                | Bool(u) -> true
                | _ -> false
                )
    | TString -> (match y with
                 | String(u) -> true
                 | _ -> false
                 )
    | TClosure -> (match y with
                   | Closure(i,e,n) -> true
                   | _ -> false
                   )
    | TRecClosure -> (match y with
                     | RecClosure(i,j,e,n) -> true
                     | _ -> false
                     )
    |TUnBound -> (match y with
                 | UnBound -> true
                 | _ -> false
                 )

(* Errori a runtime *)
exception RuntimeError of string

val typecheck : tname * evT -> bool = <fun>


exception RuntimeError of string


### Primitive

L'unica primitiva da modificare è quella della divisione.

In [28]:
(* PRIMITIVE del linguaggio *)

(* Controlla se un numero è zero *)
let is_zero(x) = match (typecheck(TInt,x),x) with
    | (true, Int(v)) -> Bool(v = 0)
    | (_, _) -> raise ( RuntimeError "Wrong type")

(* Uguaglianza fra interi *)
let int_eq(x,y) =   
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v = w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Somma fra interi *)	   
let int_plus(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v + w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Differenza fra interi *)
let int_sub(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v - w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Prodotto fra interi *)
let int_times(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v * w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")
    
(* Divisione fra interi *)
let int_div(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> 
                    if w<>0 then Int(v / w)
                            (*Anzichè chiamare la funzione raise, che fa terminare l'interprete con un errore,
                              restituiamo come risultato un valore di errore*)
                            else DivByZero 
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni di confronto *)
let less_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v < w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let greater_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v > w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni logiche *)
let bool_and(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v && w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_or(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v || w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_not(x) = 
    match (typecheck(TBool,x), x) with
    | (true, Bool(v)) -> Bool(not(v))
    | (_,_) -> raise ( RuntimeError "Wrong type")

val is_zero : evT -> evT = <fun>


val int_eq : evT * evT -> evT = <fun>


val int_plus : evT * evT -> evT = <fun>


val int_sub : evT * evT -> evT = <fun>


val int_times : evT * evT -> evT = <fun>


...

val less_than : evT * evT -> evT = <fun>


val greater_than : evT * evT -> evT = <fun>


val bool_and : evT * evT -> evT = <fun>


val bool_or : evT * evT -> evT = <fun>


val bool_not : evT -> evT = <fun>


### Interprete

In [29]:
(* Interprete *)
let rec eval (e:exp) (s:evT env) : evT = 
    match e with
    | EInt(n) -> Int(n)
    | CstTrue -> Bool(true)
    | CstFalse -> Bool(false)
    | EString(s) -> String(s)
    (*È importante sottolineare che, essendo l'ambiente definito come una funzione, 
      se s è un ambiente e x è una variabile definita, la valutazione di Den(i) 
      restituisce il valore di errore Unbound*)
    | Den(i) -> (s i)

    | Prod(e1,e2) -> int_times((eval e1 s), (eval e2 s))
    | Sum(e1, e2) -> int_plus((eval e1 s), (eval e2 s))
    | Diff(e1, e2) -> int_sub((eval e1 s), (eval e2 s))
    | Div(e1, e2) -> int_div((eval e1 s), (eval e2 s))

    | IsZero(e1) -> is_zero (eval e1 s)
    | Eq(e1, e2) -> int_eq((eval e1 s), (eval e2 s))
    | LessThan(e1, e2) -> less_than((eval e1 s),(eval e2 s))
    | GreaterThan(e1, e2) -> greater_than((eval e1 s),(eval e2 s))

    | And(e1, e2) -> bool_and((eval e1 s),(eval e2 s))
    | Or(e1, e2) -> bool_or((eval e1 s),(eval e2 s))
    | Not(e1) -> bool_not(eval e1 s)

    | IfThenElse(e1,e2,e3) -> 
        let g = eval e1 s in 
            (match (typecheck(TBool,g),g) with
            |(true, Bool(true)) -> eval e2 s
            |(true, Bool(false)) -> eval e3 s
            (*Se la guardia non è un valore booleano, ovvero se 
              typecheck(TBool, g) restituisce false, allora il valore 
              dell'espressione ifthenelse deve essere il valore di errore NonBoolGuard*)
            |(_,_) -> NonBoolGuard
            )
    | Let(i, e, ebody) -> eval ebody (bind s i (eval e s))
    | Fun(arg, ebody) -> Closure(arg,ebody,s)
    | Letrec(f, arg, fBody, leBody) ->
        let benv = bind (s) (f) (RecClosure(f, arg, fBody,s)) in
            eval leBody benv
    | Apply(eF, eArg) ->
        let fclosure = eval eF s in 
            (match fclosure with 
            | Closure(arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in 
                let aenv = bind fDecEnv arg aVal in 
                eval fbody aenv 
            | RecClosure(f, arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in
                let rEnv = bind fDecEnv f fclosure in
                let aenv = bind rEnv arg aVal in 
                eval fbody aenv
            | _ -> raise ( RuntimeError "Wrong type")
            )
    (*TryWith valuta la prima espressione, 
      se non ottiene un errore, allora restituisce il valore della prima espressione
      se ottiene un errore, che corrisponde al with, allora restituisce il valore della la seconda espressione,
      se ottiene un errore che non corrisponde al with, allora restituisce quel valore di errore*)
    | TryWith(e1, ex, e2) -> 
        let v = eval e1 s in
        match (v, ex) with 
        | UnBound,      ExVarUnbound   -> eval e2 s
        | NonBoolGuard, ExNonBoolGuard -> eval e2 s
        | DivByZero, ExDivByZero       -> eval e2 s
        | _, _ -> v;;
        

val eval : exp -> evT env -> evT = <fun>


### Esempio di esecuzione

In [31]:
(*try 
    if true then 42 else 0
with ExNonBoolGuard -> -1*)
let eCorrect = TryWith(
    IfThenElse(CstTrue,EInt(42),EInt(0)), 
    ExNonBoolGuard, 
    EInt(-1)
    );;

(*try 
    if "true" then 42 else 0
with ExNonBoolGuard -> -1*)
let eWithException = TryWith(
    IfThenElse(EString("true"),EInt(42),EInt(0)), 
    ExNonBoolGuard, 
    EInt(-1)
    );;

(*try 
    if true then 42/0 else 0
with ExDivByZero -> -1*)
let eWithUncoughtException = TryWith(
    IfThenElse(CstTrue,
                Div(EInt(42), EInt(0)),
                EInt(0)), 
    ExNonBoolGuard, 
    EInt(-1)
    );;
eval eCorrect emptyenv;; (*Mi aspetto 42*)
eval eWithException emptyenv;; (*Mi aspetto -1*)
eval eWithUncoughtException emptyenv;; (*Mi aspetto il valore di errore non catturato da ExNonBoolGuard*)

val eCorrect : exp =
  TryWith (IfThenElse (CstTrue, EInt 42, EInt 0), ExNonBoolGuard, EInt (-1))


val eWithException : exp =
  TryWith (IfThenElse (EString "true", EInt 42, EInt 0), ExNonBoolGuard,
   EInt (-1))


val eWithUncoughtException : exp =
  TryWith (IfThenElse (CstTrue, Div (EInt 42, EInt 0), EInt 0),
   ExNonBoolGuard, EInt (-1))


- : evT = Int 42


- : evT = Int (-1)


- : evT = DivByZero


Questo implementazione delle eccezioni in MiniCamel è in realtà incompleta. Dall'espressione
```
try 
    (42/0)*10
with ExDivByZero -> -1
```
Ci si aspetta di ricevere il valore `-1`, perchè viene sollevata una eccezione che dovrebbe essere catturata dal `with`. Il problema è che il valore `DivByZero`, che viene creato, non viene passato immediatamente al costrutto try, ma viene usato come risultato parziale di quale che sia la sua espressione padre. In questo caso, il valore `DivByZero` viene usato da `Prod` e moltiplicato col valore 10, ma `DivByZero` non supera il typechecking e l'interprete si ferma.

La soluzione corretta (implementata ad esmepio da NaN nell'aritmetica a virgola mobile, o dalle monadi in Haskell), sarebbe quella di riscrivere tutte le primitive per far si che, se ricevono in input un valore di errore, restituiscano in output quel valore di errore senza eseguire nessuna altra operazione.

In [32]:
(*try 
    (42/0)*10
with ExDivByZero -> -1*)
let eWithInnerException = TryWith(
    Prod(
         Div(EInt(42), EInt(0)),
         EInt(10)
        ), 
    ExDivByZero, 
    EInt(-1)
    );;

eval eWithInnerException emptyenv;;

val eWithInnerException : exp =
  TryWith (Prod (Div (EInt 42, EInt 0), EInt 10), ExDivByZero, EInt (-1))


error: runtime_error